<a href="https://colab.research.google.com/github/zaraktariq/DataStructures/blob/master/Copy_of_363_F23_Huffman_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Huffman implementation

* Given a `message`
* Find frequencies of its symbols and assign them to tree nodes, with symbol and corresponding frequency
* Add all nodes to a forest (these are tree nodes)
* While the forest has > 1 nodes:
  * Select 2 nodes with lowest frequencies
  * Remove them from forest
  * Make them L/R children of new parent node, that has no symbol but has the sum of its children frequencies as its frequency
  * Add parent to forest
* Remaining parent node is root of Huffman tree
* Traverse tree to build encoding table for each symbol, using 0 for left, 1 for right
* Use encoding table to encode (compress) the given message

In [12]:
def symbol_frequencies(message):
  """Compute the frequencies of symbols in a string

  Inputs
  ------
  message : str
    string whose symbol frequencies we must compute

  Returns
  -------
  frequencies : dict
    dictionary with symbols as keys and frequencies as values
  """
  frequencies = dict()
  # Guard against null or empty input
  if message is not None and len(message) > 0:
    for symbol in message:        # Parse input string, character by
      if symbol in frequencies:   # symbol in dictionary
        frequencies[symbol] += 1  # increase frequncy +1
      else:                       # first time I see this symbol
        frequencies[symbol] = 1   # well, freq is 1
  return frequencies              # Done (may be empty if message null/empty)

In [4]:
class Node:
  # Basic constructor
  def __init__(self, frequency, symbol=None):
    self.symbol = symbol
    self.frequency = frequency
    self.left = None
    self.right = None
  # override < operator to compare nodes based on their frequencies
  def __lt__(self, other):
    return self.frequency < other.symbol_frequency

# Your *earlier* assignment

Given the material above, create a forest of nodes with the symbols and the frequencies of a string, using the following starter code.

**We finished this in class on 10/30/23.**

For your updated assignment, **also due 11/3/23,** look at the bottom of this notebook.

In [5]:
def initialize_forest(frequencies):
  """Creates a forest of tree nodes with the symbols and their frequencies
  contained in frequency.

  Inputs
  ------
  frequency : dict
    symbols as keys, frequencies as values. e.g, for "HELLO WORLD" the frequency
    dictionary would be {'H': 1, 'E': 1, 'L': 2, 'O': 1}

  Returns
  -------
  forest : list
    list of root nodes. Each node corresponds to a symbol and its frequency
    in some message that was used to generate the input frequencies.

  """
  forest = list()
  for symbol in frequencies:
    symbol_frequency = frequencies[symbol]
    new_node = Node(symbol_frequency, symbol)
    forest.append(new_node)
  return forest

In [6]:
def smallest_node(forest):
  """Find the node in list forest, with the lowest frequency, remove it,
  and return it. The list contains Node objects -- defined earlier.
  """
  # Assume the first node has the smallest frequency
  smallest_index = 0
  # Pull the frequency(remember .frequency for the node object)
  smallest_value = forest[smallest_index].frequency
  # Traverse the rest of the list and compare
  for i in range(1, len(forest)):
    if forest[i].frequency < smallest_value:
      # Found a node with lower frequency. Update!
      smallest_index = i
      smallest_value = forest[i].frequency
  # Done. Pop (remove) and return the element at smallest_index
  return forest.pop(smallest_index)


def huffman(forest):
  """Given a forest reduce it down to 1 root using Huffamn algorithm.

  Inputs
  ------
  forest : list
    list of root nodes

  Returns
  -------
  forest : list
    the list, reduced down to one root only
  """
  while len(forest) > 1:
    # To remove two nodes with lowest frequencies, we can write an elaborate
    # method to find the two smallest values, or we can use a simple method to
    # find the smallest value, and then call the same method again!
    t1 = smallest_node(forest) # Remove note with lowest frequency
    t2 = smallest_node(forest) # Again!
    # Add frequencies of removed nodes
    combined_frequency = t1.frequency + t2.frequency
    # Create a new node with combined frequency and no symbol
    # Notice that when Node is called with just one argument (frequency),
    # the default value for sumbol is None
    new_node = Node(combined_frequency)
    # Make the two removed trees left and right children of new node
    new_node.left = t1
    new_node.right = t2
    # place new node to forest
    forest.append(new_node)
  # Done. After successive combinations of two nodes at a time, we're left
  # with only one node in the forst and that's what we return.
  return forest[0]

# Your *former* assignment

Using the methods above, derive the encoding table for a given message. For example, if the message is `HELLO WORLD`, the encoding table would be:

```text
symbol   code
L          10
O         110
W         000
D         001
E         010
R        1110
(space)  1111
```

To obtain the encoding table, traverse the Huffman tree (obtained by method ``huffman``). Most of the nodes in the tree have no symbols; just frequencies from combining nodes.

There are a few nodes however, at the bottom of the tree, that have symbols in them. The path to these nodes is a series of left and right turns from the root node, on down. Replacing a left path with '0' and a right path with '1' yields the encoding for a specific node with a symbol in it. For example, to get to the node with symbol (letter) **L**, we start at the root, go right, then left -- that's **`10`**.

In [7]:
def encoding_table(root_of_huffman):
  """Produce the encoding table from a Huffman tree.

  Inputs
  ------
  huffman_tree : node
    The root node of a Huffman tree

  Returns
  -------
  encoding_table: dictionary
    The binary code for each symbol in the tree
  """
  encoding_table = dict()
  # Initialize a stack with the root of the tree and an empty string.
  # Notice that what we are storing in the stack is a list with two components.
  # The first component is a Node object. The second component is a string that
  # will become the code for a symbol.
  stack = [[root_of_huffman, ""]]
  # As long as there are elements in the stack, pop them one at a time and
  # process them. As we pop the stack, what comes out is a list of two items:
  # the Node object and the string with the code. The Node object may or may not
  # contain a symbol. If it contains a symbol, the corresponding code is the
  # binary encoding for that symbol. If it does not contain a symbol, we follow
  # its left and right children and we update the code string with the labels
  # for left (0) and right (1).
  while stack:
    node, code = stack.pop()
    if node.symbol is not None:
      # node is leaf; save the corresponding code for this symbol
      encoding_table[node.symbol] = code
    if node.left:
      # Add the left child to the stack, together with the code enhanced by
      # the direction label (left = "0")
      stack.append([node.left, code + "0"])
    if node.right:
      # Add the right child to the stack, together with the code enhanced by
      # the direction label (right = "1")
      stack.append([node.right, code + "1"])
  # Done. Return the dictionary with the codes for all symbols.
  return encoding_table

In [8]:
# Test

import pprint # Nice print for dictionaries

message = "The unanimous Declaration of the thirteen united States of America, When in the Course of human events, it becomes necessary for one people to dissolve the political bands which have connected them with another, and to assume among the powers of the earth, the separate and equal station to which the Laws of Nature and of Nature's God entitle them, a decent respect to the opinions of mankind requires that they should declare the causes which impel them to the separation. We hold these truths to be self-evident, that all men are created equal, that they are endowed by their Creator with certain unalienable Rights, that among these are Life, Liberty and the pursuit of Happiness.--That to secure these rights, Governments are instituted among Men, deriving their just powers from the consent of the governed, --That whenever any Form of Government becomes destructive of these ends, it is the Right of the People to alter or to abolish it, and to institute new Government, laying its foundation on such principles and organizing its powers in such form, as to them shall seem most likely to effect their Safety and Happiness. Prudence, indeed, will dictate that Governments long established should not be changed for light and transient causes; and accordingly all experience hath shewn, that mankind are more disposed to suffer, while evils are sufferable, than to right themselves by abolishing the forms to which they are accustomed. But when a long train of abuses and usurpations, pursuing invariably the same Object evinces a design to reduce them under absolute Despotism, it is their right, it is their duty, to throw off such Government, and to provide new Guards for their future security.--Such has been the patient sufferance of these Colonies; and such is now the necessity which constrains them to alter their former Systems of Government. The history of the present King of Great Britain is a history of repeated injuries and usurpations, all having in direct object the establishment of an absolute Tyranny over these States. To prove this, let Facts be submitted to a candid world. He has refused his Assent to Laws, the most wholesome and necessary for the public good. He has forbidden his Governors to pass Laws of immediate and pressing importance, unless suspended in their operation till his Assent should be obtained; and when so suspended, he has utterly neglected to attend to them. He has refused to pass other Laws for the accommodation of large districts of people, unless those people would relinquish the right of Representation in the Legislature, a right inestimable to them and formidable to tyrants only. He has called together legislative bodies at places unusual, uncomfortable, and distant from the depository of their public Records, for the sole purpose of fatiguing them into compliance with his measures. He has dissolved Representative Houses repeatedly, for opposing with manly firmness his invasions on the rights of the people. He has refused for a long time, after such dissolutions, to cause others to be elected; whereby the Legislative powers, incapable of Annihilation, have returned to the People at large for their exercise; the State remaining in the mean time exposed to all the dangers of invasion from without, and convulsions within. He has endeavoured to prevent the population of these States; for that purpose obstructing the Laws for Naturalization of Foreigners; refusing to pass others to encourage their migrations hither, and raising the conditions of new Appropriations of Lands. He has obstructed the Administration of Justice, by refusing his Assent to Laws for establishing Judiciary powers. He has made Judges dependent on his Will alone, for the tenure of their offices, and the amount and payment of their salaries. He has erected a multitude of New Offices, and sent hither swarms of Officers to harrass our people, and eat out their substance. He has kept among us, in times of peace, Standing Armies without the Consent of our legislatures. He has affected to render the Military independent of and superior to the Civil power. He has combined with others to subject us to a jurisdiction foreign to our constitution, and unacknowledged by our laws; giving his Assent to their Acts of pretended Legislation: For Quartering large bodies of armed troops among us: For protecting them, by a mock Trial, from punishment for any Murders which they should commit on the Inhabitants of these States: For cutting off our Trade with all parts of the world: For imposing Taxes on us without our Consent: For depriving us in many cases, of the benefits of Trial by Jury: For transporting us beyond Seas to be tried for pretended offences For abolishing the free System of English Laws in a neighbouring Province, establishing therein an Arbitrary government, and enlarging its Boundaries so as to render it at once an example and fit instrument for introducing the same absolute rule into these Colonies: For taking away our Charters, abolishing our most valuable Laws, and altering fundamentally the Forms of our Governments: For suspending our own Legislatures, and declaring themselves invested with power to legislate for us in all cases whatsoever. He has abdicated Government here, by declaring us out of his Protection and waging War against us. He has plundered our seas, ravaged our Coasts, burnt our towns, and destroyed the lives of our people. He is at this time transporting large Armies of foreign Mercenaries to compleat the works of death, desolation and tyranny, already begun with circumstances of Cruelty & perfidy scarcely paralleled in the most barbarous ages, and totally unworthy the Head of a civilized nation. He has constrained our fellow Citizens taken Captive on the high Seas to bear Arms against their Country, to become the executioners of their friends and Brethren, or to fall themselves by their Hands. He has excited domestic insurrections amongst us, and has endeavoured to bring on the inhabitants of our frontiers, the merciless Indian Savages, whose known rule of warfare, is an undistinguished destruction of all ages, sexes and conditions. In every stage of these Oppressions We have Petitioned for Redress in the most humble terms: Our repeated Petitions have been answered only by repeated injury. A Prince whose character is thus marked by every act which may define a Tyrant, is unfit to be the ruler of a free people. Nor have We been wanting in attentions to our British brethren. We have warned them from time to time of attempts by their legislature to extend an unwarrantable jurisdiction over us. We have reminded them of the circumstances of our emigration and settlement here. We have appealed to their native justice and magnanimity, and we have conjured them by the ties of our common kindred to disavow these usurpations, which would inevitably interrupt our connections and correspondence. They too have been deaf to the voice of justice and of consanguinity. We must, therefore, acquiesce in the necessity, which denounces our Separation, and hold them, as we hold the rest of mankind, Enemies in War, in Peace Friends. We, therefore, the"
frequencies = symbol_frequencies(message)
forest = initialize_forest(frequencies)

print("Symbols and their frequencies -- this are the initial nodes in the forest")
for node in forest:
  print(node.symbol, node.frequency)

huff_tree = huffman(forest)

print("\nForest after Huffman (symbol, frequency)")

for node in forest:
  print(node.symbol, node.frequency)

print (forest[0].right.left.symbol) # What does this do? :-)

# Get the encoding table from the Huffman forest we just produced
table = encoding_table(forest[0])

print("\n\n")

# Prints the nicely formatted dictionary
pprint.pprint(table)


Symbols and their frequencies -- this are the initial nodes in the forest
T 12
h 292
e 764
  1174
u 193
n 427
a 408
i 408
m 130
o 448
s 428
D 2
c 146
l 176
r 380
t 555
f 149
d 213
S 14
A 14
, 85
W 12
C 13
v 66
b 80
y 72
p 111
w 77
g 105
q 5
L 15
N 5
' 1
G 12
k 13
. 34
- 7
R 6
H 23
M 4
j 11
F 14
P 9
z 4
; 8
x 9
B 5
O 5
K 1
J 4
: 9
Q 1
I 3
E 2
& 1

Forest after Huffman (symbol, frequency)
None 7165
None



{' ': '111',
 '&': '1011011111111',
 "'": '1011011111100',
 ',': '000011',
 '-': '1011010011',
 '.': '10111110',
 ':': '1011111110',
 ';': '1011011101',
 'A': '101101011',
 'B': '0111000000',
 'C': '011100111',
 'D': '101101111100',
 'E': '101101111101',
 'F': '101101100',
 'G': '011100110',
 'H': '01110001',
 'I': '10110111000',
 'J': '10110111101',
 'K': '1011011111101',
 'L': '101101101',
 'M': '10110111001',
 'N': '10111111111',
 'O': '0111000001',
 'P': '1011111100',
 'Q': '1011011111110',
 'R': '1011010010',
 'S': '101101010',
 'T': '011100100',
 'W': '011100101',
 'a': '0101',
 

# Your **new** assignment.

Using the methods we wrote to implement the Huffman algorithm so far:

* find a nice online text (Project Gutenberg has quite a few);
* obtain its Huffman table;
* encode the text using the Huffman table;
* compute the length of the Huffman encoded text; the length is the number of 0s and 1s you used for encoding;
* compute how many bits the text requires when using 8-bits per symbol.
* compare how much storage space you save when using Huffman.

The method ``message_from``, below, can access any URL and return it as a string -- which you can then pass to `symbol_frequencies` to get started. Make sure that the text you access is clear text -- Project Gutenberg has books available in HTML format but also in clear text. Opt for the `.txt` files.

The tasks above should be done in a new method called `compress_and_compare(message)`. The parameter `message` shall be a string.

```python
# Web stuff (for processing online text)
from urllib.request import urlopen

def message_from(url):
  """Pull an online text into a string."""
  message = ""
  text = urlopen(url)
  for line in text:
    # urlopen returns byte literals; need to decode them
    string_line = line.decode()
    message += string_line
  return message
```

In [15]:
def message_from():
  # Web stuff (for processing online text)
  from urllib.request import urlopen

  def message_from(url):
    """Pull an online text into a string."""
    message = "https://www.gutenberg.org/cache/epub/16955/pg16955-images.html"
    text = urlopen(url)
    for line in text:
      # urlopen returns byte literals; need to decode them
      string_line = line.decode()
      message += string_line
    return message

In [17]:
def compress_and_compare():
  frequencies = symbol_frequencies(message_from)
  forest = initialize_forest(frequencies)
  print("Symbols and their frequencies -- this are the initial nodes in the forest")
  for node in forest:
    print(node.symbol, node.frequency)